In [1]:
import pandas as pd #imports
import numpy as np
import multiprocessing

from fbprophet import Prophet
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as mae
from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import scipy.stats

In [2]:
def preprocess_download(data, start_date_back, end_date_back): #preprocess data
    data = data.dropna(subset=['subregion1_code'])
    data = data[data.country_code == 'US']

    data = data[['date',
                 'subregion1_code',
                 'total_deceased',
                 'average_temperature', 
                 'minimum_temperature',
                 'maximum_temperature', 
                 'rainfall', 
                 'dew_point', 
                 'relative_humidity',
                 'new_confirmed']]
    
    data['days_back'] = [(datetime.now() - day).days for day in pd.to_datetime(data.date)]
    data = data[data['days_back']<=start_date_back]
    data = data[data['days_back']>=end_date_back]
    data.fillna(0, inplace = True)
    
    res = {}
    for state in data.subregion1_code.unique():
        if state in ['MP', 
                     'AS',
                     'GU',
                     'PR', 
                     'VI']:
            continue
#         print('State: ', state)
        tmp_state = data[data.subregion1_code == state]
        tmp = []
        for date in tmp_state.date.unique():
            tmp_state_day = tmp_state[tmp_state.date == date]
            tmp.append({'ds': pd.to_datetime(date), 
                        'total_deceased': sum(tmp_state_day.total_deceased),
                        'average_temperature': np.mean(tmp_state_day.average_temperature),
                        'minimum_temperature': np.mean(tmp_state_day.minimum_temperature),
                        'maximum_temperature': np.mean(tmp_state_day.maximum_temperature),
                        'rainfall': np.mean(tmp_state_day.rainfall),
                        'dew_point': np.mean(tmp_state_day.dew_point),
                        'relative_humidity': np.mean(tmp_state_day.relative_humidity),
                        'y': sum(tmp_state_day.new_confirmed)})
        res[state] = pd.DataFrame(tmp)
    
    return(res)

In [3]:
data = preprocess_download(pd.read_csv("https://storage.googleapis.com/covid19-open-data/v2/main.csv", 
                                       low_memory=False), 
                           62, 
                           2)


In [4]:
def run_prophet(data, forecast_dist, state): # Facebook Prophet run on all states
    def mean_confidence_interval(data, confidence=0.95): #create confidence interval not for metric but to track through iteration not a true confidence interval
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return h
    
    def Prophet_reg(df): #Run Facebook Prophet
        prophet = Prophet(yearly_seasonality=False, daily_seasonality=True)
        for col in df.columns:
            if col not in ['ds','y']:
                prophet.add_regressor(col)
        prophet.fit(df)
        return prophet
    
    def make_future_dataframe(df): #make future DataFrame
        future = prophet.make_future_dataframe(periods=forecast_dist, freq='D')

        for col in df.columns:
            if col not in ['ds','y']:
                future[col] = df[col]
        future.fillna(0, inplace = True)
        return future
    
    df = data[state]
    
    y_train_1, y_test_1 = temporal_train_test_split(df, test_size=forecast_dist)  #train test split
    prophet = Prophet_reg(y_train_1)
    future = make_future_dataframe(y_train_1)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_1):]
    oot_val_1 = (y_test_1['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_2, y_test_2 = temporal_train_test_split(y_train_1, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_2)
    future = make_future_dataframe(y_train_2)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_2):]
    oot_val_2 = (y_test_2['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_3, y_test_3 = temporal_train_test_split(y_train_2, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_3)
    future = make_future_dataframe(y_train_3)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_3):]
    oot_val_3 = (y_test_3['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_4, y_test_4 = temporal_train_test_split(y_train_3, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_4)
    future = make_future_dataframe(y_train_4)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_4):]
    oot_val_4 = (y_test_4['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_5, y_test_5 = temporal_train_test_split(y_train_4, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_5)
    future = make_future_dataframe(y_train_5)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_5):]
    oot_val_5 = (y_test_5['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    ci = mean_confidence_interval([oot_val_1, oot_val_2, oot_val_3, oot_val_4, oot_val_5])
    
    prophet = Prophet_reg(df)
    future = make_future_dataframe(df)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(df):]
    pred_future = fb_preds.values[-1:][0]
    return {'State': state,
            'ci_Prediction_%': np.round(ci/pred_future*100),
            'Prediction_95_ci_l': pred_future-ci,
            'Prediction': pred_future,  
            'Prediction_95_ci_h': pred_future+ci}

In [5]:
num_cores = multiprocessing.cpu_count() # run on AWS in parallel
state_list = tqdm(list([state for state in data.keys()]))
processed_list = Parallel(n_jobs=num_cores)(delayed(run_prophet)(data, 
                                                                 3, 
                                                                 state) for state in state_list)

100%|██████████| 51/51 [00:33<00:00,  1.51it/s] 


In [6]:
state_level_predictions = pd.DataFrame(processed_list)# This Model was extremely inaccurate which pushed towards the development of the COVID IID notebook
state_level_predictions.sort_values(['Prediction'], ascending = False) # This was first iteration and model was abandoned due to erratic scoring
state_level_predictions.sort_values(['ci_Prediction_%'], ascending = False)
state_level_predictions.sort_values(['Prediction'], ascending = True)

,State,ci_Prediction_%,Prediction_95_ci_l,Prediction,Prediction_95_ci_h
14,IL,-33.0,-271935.760259,-205078.312347,-138220.864436
31,NJ,-19.0,-234660.818649,-197169.601142,-159678.383636
22,MI,-40.0,-169057.230618,-120478.094869,-71898.959120
6,CT,-39.0,-161105.556375,-116304.958762,-71504.361148
38,PA,-44.0,-143225.039321,-99192.902608,-55160.765895
24,MO,-14.0,-40731.373615,-35586.432795,-30441.491974
20,MD,-123.0,-79042.166471,-35448.894486,8144.377500
19,MA,-87.0,-58062.924596,-30984.607372,-3906.290147
47,WA,-59.0,-48896.494971,-30663.053111,-12429.611252
12,IA,-42.0,-42898.376220,-30150.686777,-17402.997335


In [7]:
state_level_predictions.to_csv('./statepreds.csv')